In [ ]:
!pip install nbmerge


  Created wheel for nbmerge: filename=nbmerge-0.0.4-py2.py3-none-any.whl size=6413 sha256=4ef36d3499af2a83e6310e3a7d33eac3ff4f1ebb622f5ad403c8596b41d904d2
  Stored in directory: /root/.cache/pip/wheels/0b/a8/31/4ae563f674b488c82ee72cfe0f86487c21532db97ae910b264
Successfully built nbmerge
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:


!nbmerge merged.ipynb randomForest.ipynb hyperparams3500for\ catboost.ipynb combined.ipynb catboost_clean.ipynb traindatacleaning-Outliers.ipynb xgb.ipynb testcat.ipynb > merged1.ipynb

In [ ]:
!pip install -r requirements.txt

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
import json
import pandas as pd
import numpy as np
import math as math
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection as model_selection
import sklearn.tree as tree
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import time
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
import catboost

from sklearn.model_selection import train_test_split

In [ ]:
data_train = pd.read_csv('Training_clean5.csv')
def root_mean_squared_log_error(y_true, y_pred):
    # Alternatively: sklearn.metrics.mean_squared_log_error(y_true, y_pred) ** 0.5
    assert (y_true >= 0).all() 
    assert (y_pred >= 0).all()
    log_error = np.log1p(y_pred) - np.log1p(y_true)  # Note: log1p(x) = log(1 + x)
    return np.mean(log_error ** 2) ** 0.5
data_test = pd.read_csv('Test_clean3.csv')

In [ ]:
#data_train['floor/stories'] = data_train.floor/data_train.stories
#data_test['floor/stories'] = data_test.floor/data_test.stories

data_train['distance'] = np.sqrt((37.6 - data_train['longitude'])**2 + (55.75 - data_train['latitude'])**2)
data_test['distance'] = np.sqrt((37.6 - data_test['longitude'])**2 + (55.75 - data_test['latitude'])**2)

#data_train.drop(['floor', 'stories', 'longitude', 'latitude'], axis=1, inplace=True)
data_train.drop(['longitude', 'latitude'], axis=1, inplace=True)
#data_test.drop(['floor', 'stories', 'longitude', 'latitude'], axis=1, inplace=True)
data_test.drop(['longitude', 'latitude'], axis=1, inplace=True)
data_train.head()


,id,seller,price,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,street,distance
0,0,3,7139520.0,59.2,12.5,31.0,2,2,1,3,...,3,9,0,1,1,1,1,0,Бунинские Луга ЖК,0.239348
1,1,0,10500000.0,88.0,14.2,48.0,18,3,1,3,...,3,25,0,1,1,1,1,0,улица Радужная,0.129713
2,2,3,9019650.0,78.5,22.5,40.8,12,3,1,3,...,3,15,0,1,1,1,0,0,Саларьево Парк ЖК,0.121183
3,3,0,10500000.0,88.0,14.0,48.0,18,3,1,3,...,3,25,0,1,1,1,1,0,улица Радужная,0.129713
4,4,1,13900000.0,78.0,17.0,35.0,7,2,1,3,...,2,15,0,1,1,1,0,0,улица Сервантеса,0.217762


In [ ]:
data_test.head()

,id,seller,area_total,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,street,distance
0,23285,3,71.4,13.16,29.44,2,1,1,3,1,...,2,20,1,1,1,1,1,0,2-я Черногрязская улица,0.052411
1,23286,1,54.0,4.00,50.00,4,1,1,3,1,...,2,15,0,1,1,0,0,0,улица Сервантеса,0.216436
2,23287,3,39.4,19.60,11.20,10,1,1,3,1,...,3,9,0,1,1,1,1,0,Саларьево Парк ЖК,0.228879
3,23288,3,60.9,15.68,55.56,5,2,1,3,1,...,2,17,1,1,0,0,1,3,улица Александры Монаховой,0.217394
4,23289,3,34.0,9.00,17.00,14,1,1,3,1,...,2,17,0,1,1,1,1,0,улица 3-я Нововатутинская,0.345584


In [ ]:
target_train = data_train.price/data_train.area_total
y_train_area_total = data_train.area_total
y_test_area_total = data_test.area_total
data_train.drop(['price', 'area_total'], axis=1, inplace=True)
data_train.head()
data_test.drop(['area_total'], axis=1, inplace=True)


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ord_enc = OrdinalEncoder()
data_train['street'] = ord_enc.fit_transform(data_train[["street"]]).astype(int)
data_train.head()

,id,seller,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,bathrooms_private,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,street,distance
0,0,3,12.5,31.0,2,2,1,3,0,2,...,3,9,0,1,1,1,1,0,267,0.239348
1,1,0,14.2,48.0,18,3,1,3,2,0,...,3,25,0,1,1,1,1,0,1561,0.129713
2,2,3,22.5,40.8,12,3,1,3,0,2,...,3,15,0,1,1,1,0,0,774,0.121183
3,3,0,14.0,48.0,18,3,1,3,0,2,...,3,25,0,1,1,1,1,0,1561,0.129713
4,4,1,17.0,35.0,7,2,1,3,1,0,...,2,15,0,1,1,1,0,0,1589,0.217762


In [ ]:
import lightgbm

In [ ]:
X_train1, X_val1, y_train, y_val = train_test_split(data_train, target_train, test_size = 0.2, random_state=42)

X_train1.shape, X_val1.shape, y_train.shape, y_val.shape
X_train = X_train1.drop(['id'], axis=1)
X_val = X_val1.drop(['id'], axis=1)

In [ ]:


#cat_feature
#cat_feature_indices = np.array([ 3,6,10,15,16,20,21,24,25])

In [ ]:
train_data = lightgbm.Dataset(X_train, label = y_train)
test_data = lightgbm.Dataset(X_val, label = y_val)

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'dart',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 10000
}

In [ ]:
gbm = lightgbm.LGBMRegressor(**hyper_params)

In [ ]:
X_train.drop(['layout', 'ceiling', 'windows_court', 'new','elevator_passenger', 'elevator_service', 'garbage_chute', 'heating'], axis=1, inplace=True)

In [ ]:
X_val.drop(['layout', 'ceiling', 'windows_court', 'new','elevator_passenger', 'elevator_service', 'garbage_chute', 'heating'], axis=1, inplace=True)

In [ ]:
gbm.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='RMSE', early_stopping_rounds=1000)

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9487]	valid_0's rmse: 51611.6	valid_0's l1: 25231.9	valid_0's l2: 2.66376e+09
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9488]	valid_0's rmse: 51612.8	valid_0's l1: 25232.4	valid_0's l2: 2.66388e+09
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9489]	valid_0's rmse: 51614	valid_0's l1: 25233.9	valid_0's l2: 2.664e+09
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9490]	valid_0's rmse: 51611.4	valid_0's l1: 25234.5	valid_0's l2: 2.66373e+09
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9491]	valid_0's rmse: 51608.5	valid_0's l1: 25232.8	valid_0's l2: 2.66344e+09
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9492]	valid_0's rmse: 51609.5	valid_0's l1: 25234.3	valid_0's l2: 2.66354e+09
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[9493

LGBMRegressor(bagging_fraction=0.8, bagging_freq=10, boosting_type='dart',
              feature_fraction=0.8, learning_rate=0.05, max_bin=512,
              max_depth=8, metric=['l1', 'l2'], num_iterations=10000,
              num_leaves=128, objective='regression', task='train', verbose=0)

In [ ]:
y_pred = gbm.predict(X_train, num_iteration=gbm.best_iteration_)

In [ ]:
from sklearn.metrics import mean_squared_log_error 

In [ ]:
# Basic RMSE
print('The rmse of prediction is:', round(mean_squared_log_error(y_pred, y_train) ** 0.5, 5))

y_val_pred = gbm.predict(X_val, num_iteration=gbm.best_iteration_)
print('The rmse of prediction is:', round(mean_squared_log_error(y_val_pred, y_val) ** 0.5, 5))

#The rmse of prediction is: 0.01839
#The rmse of prediction is: 0.15097

The rmse of prediction is: 0.02946
The rmse of prediction is: 0.13968


In [ ]:

data_test['street'] = ord_enc.fit_transform(data_test[["street"]]).astype(int)
data_test.head()
#data_test


,id,seller,area_kitchen,area_living,floor,rooms,layout,ceiling,bathrooms_shared,bathrooms_private,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,street,distance
0,23285,3,13.16,29.44,2,1,1,3,1,1,...,2,20,1,1,1,1,1,0,50,0.052411
1,23286,1,4.00,50.00,4,1,1,3,1,1,...,2,15,0,1,1,0,0,0,1112,0.216436
2,23287,3,19.60,11.20,10,1,1,3,1,0,...,3,9,0,1,1,1,1,0,547,0.228879
3,23288,3,15.68,55.56,5,2,1,3,1,0,...,2,17,1,1,0,0,1,3,856,0.217394
4,23289,3,9.00,17.00,14,1,1,3,1,0,...,2,17,0,1,1,1,1,0,818,0.345584


In [ ]:
data_test1 = data_test.drop(["id",'layout', 'ceiling', 'windows_court', 'new','elevator_passenger', 'elevator_service', 'garbage_chute', 'heating'],axis=1)
print(data_test1)
y_test_pred = gbm.predict(data_test1, num_iteration = gbm.best_iteration_)

      seller  area_kitchen  area_living  floor  rooms  bathrooms_shared  \
0          3         13.16        29.44      2      1                 1   
1          1          4.00        50.00      4      1                 1   
2          3         19.60        11.20     10      1                 1   
3          3         15.68        55.56      5      2                 1   
4          3          9.00        17.00     14      1                 1   
...      ...           ...          ...    ...    ...               ...   
9932       3         19.90        56.70     16      3                 1   
9933       1          8.88        33.22      3      3                 2   
9934       1         22.06        78.22     15      1                 1   
9935       1         10.50        15.10     14      1                 1   
9936       1          7.00        49.00      2      3                 0   

      bathrooms_private  windows_street  balconies  loggias  ...  phones  \
0                     1

In [ ]:
final_labels = y_test_pred * y_test_area_total
final_labels.head()

0    2.814662e+07
1    1.179517e+07
2    6.097245e+06
3    9.359757e+06
4    5.126253e+06
Name: area_total, dtype: float64

In [ ]:
submission = pd.DataFrame()
submission['id'] = data_test.id
submission['price_prediction'] = final_labels # Predict on non-nan entries
 # Fill missing entries with mean predictor
print(f'Generated {len(submission)} predictions')

# Export submission to csv with headers
#submission.to_csv('sample_submission.csv', index=False)

submission.to_csv('sample_submission17.csv', index=False)

Generated 9937 predictions


In [ ]:
gbm.booster_.feature_importance()
fea_imp_ = pd.DataFrame({'cols':X_train.columns, 'fea_imp':gbm.feature_importances_})
fea_imp_.loc[fea_imp_.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)

,cols,fea_imp
20,distance,139107
2,area_living,115698
12,building_id,94181
1,area_kitchen,91902
14,constructed,91826
19,street,80956
16,stories,74534
3,floor,67646
13,district,46946
10,condition,29160


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=99c1e030-46bc-40fa-835c-0661eae0d488' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>